In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
submission = pd.read_csv('/kaggle/input/titanic/gender_submission.csv')

train.head()  # cf) tail()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SimpleNN(nn.Module): # nn.Module을 상속받는 SimpleNN 이름의 신경망 클래스 정의
    def __init__(self):
        super().__init__()  # 부모 클래스의 생성자 호출
        self.classifier = nn.Sequential( # 순차적인 레이어를 담는 컨테이너 생성
            nn.Linear(5, 128), # 입력층 5개에서 은닉층 128개로 연결
            ## Batch Normalization between 'Layer' and 'Activation function'
            ## 배치 정규화: 각 레이어의 활성화 함수 전에 적용, 학습 안정성과 속도 향상
            nn.BatchNorm1d(128),  # 128개의 뉴런에 대해 1D 배치 정규화 적용
            nn.ReLU(),
            ## Drop out after 'Activation function'
            ## 드롭아웃: 과적합을 방지하기 위해 활성화 함수 후에 적용, 일부 뉴런을 무작위로 0으로 설정
            nn.Dropout(0.1),  # 10% 드롭아웃 적용
            nn.Linear(128, 256),  # 128개의 뉴런에서 256개의 뉴런으로 연결
            nn.BatchNorm1d(256), 
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(128, 1),  # 128개의 뉴런에서 하나의 출력 뉴런으로 연결
            nn.Sigmoid()  # Sigmoid 활성화 함수 적용→이진 분류를 위한 출력
        )

    def forward(self, x):  # 순전파 함수 정의
        x = x.view(x.size(0), -1)  # .size(0): 텐서 x의 첫 번째 차원의 크기 반환, 보통 배치 크기(batch size)
        x = self.classifier(x)  # 정의된 순차 컨테이너(nn.Sequential)를 통해 입력 데이터 x를 전달
        return x

In [7]:
# 훈련 데이터와 테스트 데이터를 결합
data_set = pd.concat((train.drop(['Survived'], axis = 1), test), axis = 0)

# 분석에 불필요한 열 제거
data_set = data_set.drop(['PassengerId', 'Name', 'Sex', 'Ticket', 'Cabin', 'Embarked'], axis = 1)
# 결측치를 해당 열의 평균값으로 채움
data_set = data_set.fillna(data_set.mean())


# 원래의 훈련 데이터셋 크기를 기준으로 훈련 및 테스트 데이터셋을 다시 분리
n_train = train.shape[0]
train_x, test_x = data_set[:n_train], data_set[n_train:]
# 훈련 데이터셋에서 'Survived' 열을 레이블로 사용
train_y = train['Survived']

# 데이터프레임을 넘파이 배열로 변환
train_x = train_x[train_x.keys()].values
test_x = test_x[test_x.keys()].values
train_y = train_y.values

import torch.optim as optim
from torch.autograd import Variable

simple_nn = SimpleNN()
optimizer = optim.Adam(simple_nn.parameters(), lr=0.01)
error = nn.BCELoss()

batch_size = 99
# 배치의 개수 계산
batch_count = int(len(train_x) / batch_size)

for epoch in range(300):
    train_loss = 0
    num_right = 0
    # 각 배치별로 모델 훈련
    for i in range(batch_count):
        start = i * batch_size
        end = start + batch_size
        # 훈련 데이터와 레이블을 텐서로 변환
        tensor_x = torch.FloatTensor(train_x[start:end])
        tensor_y = torch.FloatTensor(train_y[start:end]).reshape(-1, 1)
        
        # 옵티마이저의 그라디언트 초기화
        optimizer.zero_grad()
        # 모델 통해 예측값 계산
        output = simple_nn(tensor_x)
        # 손실 계산
        loss = error(output, tensor_y)
        # 손실에 대한 그라디언트를 계산하고, 옵티마이저를 사용해 매개변수를 업데이트
        loss.backward()
        optimizer.step()
        
        # 배치에 대한 총 손실 업데이트
        train_loss += loss.item() * batch_size
        
        # 예측 결과와 실제 레이블을 비교하여 정확도 계산
        result = [1 if out >= 0.5 else 0 for out in output]
        num_right += np.sum(np.array(result) == train_y[start:end])
    
    # 에폭마다 평균 손실과 정확도를 계산
    train_loss = train_loss / len(train_x)
    accuracy = num_right / len(train_x)
    
    # 매 25 에폭마다 현재의 손실과 정확도를 출력
    if epoch % 25 == 0:
        print('Loss: {} Accuracy: {}% Epoch:{}'.format(train_loss, accuracy, epoch))

print('Training Ended')

Loss: 0.6448364721404182 Accuracy: 0.6520763187429854% Epoch:0
Loss: 0.5292045937644111 Accuracy: 0.7362514029180696% Epoch:25
Loss: 0.4643124242623647 Accuracy: 0.7699214365881033% Epoch:50
Loss: 0.41540297203593785 Accuracy: 0.8181818181818182% Epoch:75
Loss: 0.39630649818314445 Accuracy: 0.8260381593714927% Epoch:100
Loss: 0.33413348264164394 Accuracy: 0.8473625140291807% Epoch:125
Loss: 0.3121591541502211 Accuracy: 0.8630751964085297% Epoch:150
Loss: 0.28715767628616756 Accuracy: 0.877665544332211% Epoch:175
Loss: 0.2581160184409883 Accuracy: 0.8933782267115601% Epoch:200
Loss: 0.25267959468894535 Accuracy: 0.9012345679012346% Epoch:225
Loss: 0.2591664277844959 Accuracy: 0.8922558922558923% Epoch:250
Loss: 0.23605208264456856 Accuracy: 0.8911335578002245% Epoch:275
Training Ended


In [8]:
# test_x를 FloatTensor로 변환
tensor_test_x = torch.FloatTensor(test_x)
# 평가 중에는 gradient 계산이 필요 없으므로, no_grad() 컨텍스트 사용
with torch.no_grad():
    # 학습된 신경망 모델을 사용하여 test_x에 대한 예측 수행
    test_output = simple_nn(tensor_test_x)
    # 예측 결과를 이진 분류로 변환합니다.
    # 출력 값이 0.5 이상이면 1로, 그렇지 않으면 0으로 변환
    result = np.array([1 if out >= 0.5 else 0 for out in test_output])
    # 예측 결과를 데이터프레임으로 생성
    # 'PassengerId'와 'Survived' 열을 포함하는 데이터프레임을 생성
    submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': result})
    # 결과를 'submission.csv' 파일로 저장
    # index=False 옵션: 인덱스를 포함x
    submission.to_csv('submission.csv', index=False)